In [ ]:
import flika_JSON_IO as flikaIO
import feature_calculator as fcalc
import numpy as np
import os
import pandas as pd

In [ ]:
working_dir = r'/home/vivek/Documents/Python Programs/piezo1/Piezo1_Rg_Overlays/JSON_example_file/'
working_file = 'GB_221_Halo_B2.json'
experiment_name = 'GB_221_Halo_B2'
minfrm = 200
filePath = os.path.join(working_dir, working_file)

In [ ]:
GB_221_B2_tracks = flikaIO.json_to_pandas(filePath, experiment_name, minfrm)
GB_221_B2_tracks['ID'] = GB_221_B2_tracks['ID'].astype(int)

In [ ]:
print(GB_221_B2_tracks.loc[(GB_221_B2_tracks['ID'] == 0) & (GB_221_B2_tracks['Frame'] <= 2)][['X', 'Y']])

In [ ]:
Rg_lastFrame = fcalc.radGyr_lastFrame(GB_221_B2_tracks)

In [ ]:
print(Rg_lastFrame)

In [ ]:
print(GB_221_B2_tracks)

In [ ]:
GB_221_B2_tracks['Rg'] = np.nan
for trackID in np.arange(0, len(Rg_lastFrame), 1):
    GB_221_B2_tracks.loc[GB_221_B2_tracks.ID == trackID, 'Rg'] = Rg_lastFrame[trackID]
print(GB_221_B2_tracks)